In [1]:
import MySQLdb

In [2]:
db = MySQLdb.connect(host="localhost",  # host      # 127.0.0.1
                     user="root",       # username  # root
                     passwd="1234",     # password  # 1234
                     db="naver_movie")         # DB NAME   

In [3]:
db

<_mysql.connection open to 'localhost' at 46fc508>

In [4]:
# 데이터베이스 커서(Cursor)는 일련의 데이터에 
# 순차적으로 액세스할 때 검색 및 "현재 위치"를 
# 포함하는 데이터 요소이다.
cur = db.cursor()
# https://movie.naver.com/movie/bi/mi/basic.nhn?code=161967

In [5]:
# IF(조건, 참값, 거짓값)
query = """
        CREATE TABLE IF NOT EXISTS naver_movie
            (code    INTEGER      NOT NULL      PRIMARY KEY,
             created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
             title   VARCHAR(100) NOT NULL,
             story   TEXT,
             open_date  DATE)
             ENGINE INNODB;
        """
cur.execute(query)


0

In [7]:
query = """
        CREATE TABLE IF NOT EXISTS movie_comment(
            score    INTEGER    NOT NULL,
            content  TEXT       NOT NULL,
            movie_code INTEGER,
            writer   VARCHAR(100),
            write_date DATETIME,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (writer, write_date),
            FOREIGN KEY(movie_code)
            REFERENCES naver_movie(code) on delete cascade)
            ENGINE INNODB;
"""
cur.execute(query)

0

## 영화 크롤링

In [27]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

In [28]:
MOVIE_LIST_URL = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'

# ?sel=pnt&date=20190616

In [35]:
def get_movie_list(start_page=1, end_page=1):
    URL = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn"
    params = {
        'sel':'pnt',
        'date':20190616
    }
    
    movie_list = []
    for i in range(start_page, end_page+1):
        params['page'] = i
        
        resp = requests.get(URL, params)
        soup = BeautifulSoup(resp.content)
        
        list_ranking = soup.find(class_='list_ranking').find_all('tr')
        for movie_tr in list_ranking:
            try:
                code = movie_tr.find('a')['href'].split('=')[-1]
                title = movie_tr.find('a').text
            
                movie_list.append({
                    'code':code,
                    'title':title
                })
            except Exception:
                pass
            
    return movie_list

In [36]:
def get_movie_info(movie):
    code = movie.get('code')
    title = movie.get('title')
    
    base_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
    params = {
        'code': code,
    }
    
    resp = requests.get(base_url, params)
    soup = BeautifulSoup(resp.content)
    
    open_date_tag = soup.find('dl', class_='info_spec').find('dd').find_all('span')[-1]

    open_date = open_date_tag.text
    story = soup.find('div', class_='story_area')
    
    movie['story'] = story
    movie['open_date'] = open_date
    
    return movie

In [37]:
movie_list = get_movie_list()
movie_list[0]

{'code': '171539', 'title': '그린 북'}

In [ ]:
get_movie_info(movie_list[0])

In [ ]:
def 

In [ ]:
# https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20190616

In [8]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
import datetime
import math

In [10]:
# movie_list

# 한 화면에 50페이지
PAGE_SIZE = 50
MOVIE_LENGTH = 100
pages = math.ceil(MOVIE_LENGTH/PAGE_SIZE)
pages

2

In [11]:
def get_movie_list(start_page, end_page):
    today = datetime.datetime.now()
    
    movie_list = []
    for page in range(start_page, end_page+1):
        movie_list_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'
        params = {
            'sel':'pnt',
            'date':today.strftime('%Y%m%d'),
            'page':page
        }
        resp = requests.get(movie_list_url)
        resp.encoding = 'utf-8'
    
        soup = BeautifulSoup(resp.content)
        # list_ranking = soup.find(class_='list_ranking').find_all('tr')
        list_ranking = soup.select('.list_ranking>tbody>tr')
    
        for movie_tr in list_ranking:
            all_td = movie_tr.find_all('td')
            try:
                rank = all_td[0].img['alt']
                link = all_td[1].find('a')['href']
                title = all_td[1].find('a').text
                
                code = parse.parse_qs(parse.urlsplit(link).query)['code'][0]
            except TypeError:
                pass

            else:
                movie_list.append({
                    'img':img,
                    'link':link,
                    'title':title,
                    'code':code
                })
                
    return movie_list
    
    

In [12]:
def get_movie_detail(code):
    base_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
    params = {
        'code':code
    }
    resp = requests.get(base_url, params)
    resp.encoding ='utf-8'
    soup = BeautifulSoup(resp.content)
    
    opend = soup.find('dt', text='개요()').find_next_sibling('dd').find_all('span')[-1].text.strip()
    opend = ''.join(opend.split('\n'))[:10]

    story = soup.find('div', class_='story_area')
    print(story)
    return {
        'opend':opend,
        'story':story
    }
    
    
def get_movie_comment(code, last_comment='15788558'):
    base_url = '/movie/bi/mi/pointWriteFormList.nhn'
    page = 1
    comment_list = []
    
    while True:
        params={
            'code':171539,
            'type':'after',
            'order':'newest',
            'page':page
        }
        resp = requests.get(base_url, params)
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.content)
        
        comments = soup.find('div', class_='score_result').find_all('li')
        for comment in comments:
            star = comment.find('div', class_='star_score').text
            reple = comment.find('div', class_='score_reple').find('p').text
            
            dd = comment.find('div', class_='score_reple').find('dl').find_all('dd')
            user = dd[0]
            created_at = dd[1]
            
            comment_id = comment.find('div', class_='btn_area').find_all('span')[-1]['class'][0].split('_')[1]
            
            if int(comment_id) <= int(last_comment):
                return comment_list
            
            # 실습용 코드
            if page ==3:
                break
            
            comment_list.append({
                'star':star,
                'reple':reple,
                'user': user,
                'created_at': created_at,
                'comment_id': comment_id
            })
            
        
        page += 1
    return comment_list
    
    
    
    

In [13]:
movie_list = get_movie_list(1, pages)

In [16]:
movie_list

[{'img': '01',
  'link': '/movie/bi/mi/basic.nhn?code=161967',
  'title': '기생충',
  'code': '161967'},
 {'img': '02',
  'link': '/movie/bi/mi/basic.nhn?code=163788',
  'title': '알라딘',
  'code': '163788'},
 {'img': '03',
  'link': '/movie/bi/mi/basic.nhn?code=97631',
  'title': '맨 인 블랙: 인터내셔널',
  'code': '97631'},
 {'img': '04',
  'link': '/movie/bi/mi/basic.nhn?code=164125',
  'title': '엑스맨: 다크 피닉스',
  'code': '164125'},
 {'img': '05',
  'link': '/movie/bi/mi/basic.nhn?code=177371',
  'title': '0.0MHz',
  'code': '177371'},
 {'img': '06',
  'link': '/movie/bi/mi/basic.nhn?code=181704',
  'title': '로켓맨',
  'code': '181704'},
 {'img': '07',
  'link': '/movie/bi/mi/basic.nhn?code=18781',
  'title': '이웃집 토토로',
  'code': '18781'},
 {'img': '08',
  'link': '/movie/bi/mi/basic.nhn?code=136900',
  'title': '어벤져스: 엔드게임',
  'code': '136900'},
 {'img': '09',
  'link': '/movie/bi/mi/basic.nhn?code=183836',
  'title': '천로역정: 천국을 찾아서',
  'code': '183836'},
 {'img': '010',
  'link': '/movie/bi/mi/basi

In [24]:
def insert_movie(movie):
    code = movie.get('code')
    title = movie.get('title')
    open_date = movie.get('open_date')
    
    query = """
    INSERT INTO naver_movie
    (code, title, open_date)
    VALUES('{}', '{}', '{}')
    """.format(code, title, open_date)
    
    cur.execute(query)
    
def insert_comment(movie_code, comment):
    # movie_comment

    write = comment.get('writer')
    score = comment.get('score')
    wrter = comment.get('writer')
    write_date = comment.get('write_date')
    content = comment.get('content')
    
    query = """
    INSERT INTO movie_comment
    (score, content, writer, write_date, movie_code)
    VALUES('{}', '{}', '{}')
    """.format(score, content, writer, write_date, movie_code)
    
    cur.execute(query)
    

In [25]:
insert_movie({
    'code': '12',
    'title': 'ddd',
    'open_date': '2011.01.12',
})

OperationalError: (1054, "Unknown column 'open_date' in 'field list'")

In [ ]:
for movie in movie_list:
    detail = get_movie_detail(movie['code'])
    
    open_date = detail['opend']
    story = detail['story']
    comment_list = get_movie_comment(movie['code'])

In [18]:
resp = requests.get(movie_list_url)
resp

NameError: name 'movie_list_url' is not defined

In [ ]:
# INSERT INTO table (column_list)
# VALUES (value_list)
# ON DUPLICATE KEY UPDATE
#    c1 = v1, 
#    c2 = v2,
#    ...;

In [26]:
cur.close()